In [ ]:
# https://plotly.com/python/maps/

In [ ]:
#!bash
!pip uninstall geojson
!pip uninstall plotly

In [ ]:
!pip install geojson
!pip install plotly

**Data Cleaning**

In [64]:
import pandas as pd



In [65]:
from urllib.request import urlopen
import json
import pandas as pd

In [66]:


df = pd.read_csv("https://raw.githubusercontent.com/moon1ock/AQI_vs_Corona/main/data/daily_aqi_by_county_2020.csv")

df.head()

,State Name,county name,State Code,County Code,FIPS,Date,AQI,Category,Defining Parameter,Defining Site,Number of Sites Reporting
0,AL,Baldwin county,1,3,13.0,1/1/20,48,Good,PM2.5,01-003-0010,1
1,AL,Baldwin county,1,3,NaN,1/4/20,13,Good,PM2.5,01-003-0010,1
2,AL,Baldwin county,1,3,NaN,1/7/20,14,Good,PM2.5,01-003-0010,1
3,AL,Baldwin county,1,3,NaN,1/10/20,39,Good,PM2.5,01-003-0010,1
4,AL,Baldwin county,1,3,NaN,1/13/20,29,Good,PM2.5,01-003-0010,1


In [67]:

#Generate a new variable FIPS: "0XX" (State Code) + "00X" (County Code)

df["County Code"] = df["County Code"].apply(lambda x:'{0:0>3}'.format(x))
df["State Code"] = df["State Code"].apply(lambda x:'{0:0>2}'.format(x))


df['County Code']=df['County Code'].astype(str)
df['State Code']=df['State Code'].astype(str)
df['fips'] = df[['State Code', 'County Code']].apply(lambda x: ''.join(x), axis=1)

#Sorting by county and calculating average AQI for each 
df.groupby(['fips']).mean()


,FIPS,AQI,Number of Sites Reporting
fips,,,
01003,13.0,35.392857,1.000000
01027,NaN,27.379310,1.000000
01049,NaN,36.925620,1.243802
01051,NaN,31.637584,1.000000
01055,NaN,34.994083,1.289941
...,...,...,...
72113,NaN,19.616379,1.000000
72127,NaN,15.868132,1.000000
78010,NaN,24.107143,1.000000


In [68]:
# COVID Data
import json 
import urllib.request  

df2 = pd.read_csv("https://covidtracking.com/data/download/all-states-history.csv")

df = df2[['date', 'state', 'deathConfirmed', 'positive', 'recovered']]
df2[['date']] =  df[['date']].apply(pd.to_datetime)
df2.tail()



,date,state,dataQualityGrade,death,deathConfirmed,deathIncrease,deathProbable,hospitalized,hospitalizedCumulative,hospitalizedCurrently,hospitalizedIncrease,inIcuCumulative,inIcuCurrently,negative,negativeIncrease,negativeTestsAntibody,negativeTestsPeopleAntibody,negativeTestsViral,onVentilatorCumulative,onVentilatorCurrently,positive,positiveCasesViral,positiveIncrease,positiveScore,positiveTestsAntibody,positiveTestsAntigen,positiveTestsPeopleAntibody,positiveTestsPeopleAntigen,positiveTestsViral,recovered,totalTestEncountersViral,totalTestEncountersViralIncrease,totalTestResults,totalTestResultsIncrease,totalTestsAntibody,totalTestsAntigen,totalTestsPeopleAntibody,totalTestsPeopleAntigen,totalTestsPeopleViral,totalTestsPeopleViralIncrease,totalTestsViral,totalTestsViralIncrease
15796,2020-01-24,WA,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,0.0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0
15797,2020-01-23,MA,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,2.0,1,NaN,NaN,NaN,NaN,NaN,0,2.0,1
15798,2020-01-23,WA,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,0.0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0
15799,2020-01-22,MA,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,1.0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,1
15800,2020-01-22,WA,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,0.0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0


In [69]:
# Covid data per week by State
df2['date'] = pd.to_datetime(df2['date']) - pd.to_timedelta(7, unit='d')
df_weekly = df2.groupby(['state', pd.Grouper(key='date', freq='W-MON')])['positive'].sum().reset_index().sort_values(by = ['state','date'])
df_weekly.to_csv('covid_weekly.csv', sep=',')
df_weekly.head()

,state,date,positive
0,AK,2020-03-02,0.0
1,AK,2020-03-09,0.0
2,AK,2020-03-16,114.0
3,AK,2020-03-23,547.0
4,AK,2020-03-30,1125.0


In [70]:
#Daily AI data by State that we will convert into weekly ant take the mean
df2 = pd.read_csv("https://raw.githubusercontent.com/moon1ock/AQI_vs_Corona/main/data/initial_daily_aqi_by_state_2020%20(1).csv")
df2 = df2[['State Name', 'Date', 'AQI']]
df2.head()

,State Name,Date,AQI
0,AL,1/1/20,49
1,AL,1/2/20,33
2,AL,1/3/20,42
3,AL,1/4/20,64
4,AL,1/5/20,53


In [71]:
# fixing date format inconsistencies
for index, row in df2.iterrows():
  ind =  len(row['Date'])
  if row["Date"][ind - 3] == '/':
    df2['Date'][index] = row['Date'] + '20'
df2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,State Name,Date,AQI
0,AL,1/1/2020,49
1,AL,1/2/2020,33
2,AL,1/3/2020,42
3,AL,1/4/2020,64
4,AL,1/5/2020,53
...,...,...,...
12036,WY,6/27/2020,64
12037,WY,6/28/2020,45
12038,WY,6/29/2020,46
12039,WY,6/30/2020,53


In [72]:
# Turning AQI data into weekly by state
df2['Date'] = pd.to_datetime(df2['Date']) - pd.to_timedelta(7, unit='d')
df2 = df2.groupby(['State Name', pd.Grouper(key='Date', freq='W-MON')])['AQI'].mean().reset_index().sort_values(by = ['State Name','Date'])
df2 = df2.rename(columns = {'Date': 'date', 'State Name': 'state'}, inplace = False)#changing column names to match the COVID columns
df2.to_csv('AQI_weekly.csv', sep=',')
df2.head()


,state,date,AQI
0,AK,2019-12-30,54.166667
1,AK,2020-01-06,53.142857
2,AK,2020-01-13,48.714286
3,AK,2020-01-20,38.571429
4,AK,2020-01-27,33.571429


In [73]:
#Merging COVID and AQI data into one by state name and date
import numpy as np
df2 = pd.merge(df_weekly, df2, on=['state', 'date'])


df2

,state,date,positive,AQI
0,AK,2020-03-02,0.0,20.000000
1,AK,2020-03-09,0.0,39.428571
2,AK,2020-03-16,114.0,31.000000
3,AK,2020-03-23,547.0,32.571429
4,AK,2020-03-30,1125.0,24.428571
...,...,...,...,...
1344,WY,2020-06-01,6527.0,43.285714
1345,WY,2020-06-08,7175.0,54.285714
1346,WY,2020-06-15,8126.0,46.142857
1347,WY,2020-06-22,9489.0,46.857143
